<a href="https://colab.research.google.com/github/Heal-me-KI/KI_Task/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nama : Hilmy Ismail Hidayat**
# **NPM : 2205048575**

**Full Code**

In [12]:
import torch
import cv2
import numpy as np
import warnings

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # YOLOv5 small model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load video
video_path = '/content/drive/My Drive/road_trafifc.mp4'
cap = cv2.VideoCapture(video_path)

object_tracks = []  # List to store tracking data

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# First pass: Detect and store object tracks
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform YOLO detection
    results = model(frame)
    boxes = results.xyxy[0].cpu().numpy()  # Bounding boxes in xyxy format

    frame_tracks = []  # Temporary storage for current frame detections
    frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Get current frame ID

    for box in boxes:
        x1, y1, x2, y2, conf, cls = box  # Unpack bounding box
        label = model.names[int(cls)]  # Get class label

        # Store tracking data as (frame_id, label, x1, y1, x2, y2, confidence)
        frame_tracks.append((frame_id, label, int(x1), int(y1), int(x2), int(y2), conf))

    object_tracks.append(frame_tracks)  # Append current frame's tracks

cap.release()  # Release the video capture object

# Normalize object tracking data
normalized_tracks = []
for frame in object_tracks:
    normalized_frame = [
        (
            obj[0],  # Frame ID
            obj[1],  # Label
            obj[2] / frame_width, obj[3] / frame_height,  # Normalized x1, y1
            obj[4] / frame_width, obj[5] / frame_height,  # Normalized x2, y2
            obj[6]  # Confidence
        )
        for obj in frame
    ]
    normalized_tracks.append(normalized_frame)

# Second pass: Write annotated video output
cap = cv2.VideoCapture(video_path)  # Reinitialize video capture
output_path = '/content/drive/My Drive/road_traffic_output_.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Retrieve the detections for the current frame
    frame_tracks = object_tracks[frame_idx] if frame_idx < len(object_tracks) else []

    for obj in frame_tracks:
        _, label, x1, y1, x2, y2, conf = obj  # Unpack object info

        # Draw bounding box and label on frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write frame with detections to output video
    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

print(f"Output video saved to {output_path}")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-2 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output video saved to /content/drive/My Drive/road_traffic_output_.mp4


**Result**

In [11]:
# Print the final normalized tracking data
print(normalized_tracks)

[[(1, 'person', 0.14375, 0.5111111111111111, 0.2703125, 0.95, 0.89774746), (1, 'bus', 0.384375, 0.48333333333333334, 0.5109375, 0.6027777777777777, 0.80263704), (1, 'backpack', 0.196875, 0.5833333333333334, 0.2671875, 0.7416666666666667, 0.61541957), (1, 'truck', 0.303125, 0.5305555555555556, 0.3765625, 0.6166666666666667, 0.52057433), (1, 'person', 0.3984375, 0.5444444444444444, 0.41875, 0.6222222222222222, 0.50805885), (1, 'truck', 0.934375, 0.5333333333333333, 0.9984375, 0.6638888888888889, 0.4974846), (1, 'person', 0.36875, 0.5472222222222223, 0.3921875, 0.6277777777777778, 0.46563628), (1, 'person', 0.428125, 0.5361111111111111, 0.45625, 0.6194444444444445, 0.44428134), (1, 'person', 0.090625, 0.5722222222222222, 0.115625, 0.6833333333333333, 0.43011373), (1, 'umbrella', 0.0, 0.45, 0.1359375, 0.5416666666666666, 0.40476546), (1, 'motorcycle', 0.36875, 0.5472222222222223, 0.3921875, 0.6277777777777778, 0.35706884), (1, 'backpack', 0.146875, 0.5277777777777778, 0.26875, 0.9444444444